In [31]:
# example of defining composite models for training cyclegan generators
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.initializers import RandomNormal
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import os
from PIL import Image
from matplotlib import image
import numpy as np
from numpy import ones
from numpy import zeros

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_image = Input(shape=image_shape)
	nfilters = 32
	kernel_size = 4
	# C64
	d = Conv2D(nfilters, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(nfilters*2, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(nfilters*4, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(nfilters*8, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(nfilters*8, kernel_size, padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, kernel_size, padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	return model

# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	n_filters = 32
	kernel_size = 3
	# c7s1-64
	g = Conv2D(n_filters, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(n_filters*2, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(n_filters*4, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(n_filters*4, g)
	# u128
	g = Conv2DTranspose(n_filters*2, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(n_filters, kernel_size, strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(1, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model

In [32]:
from random import randint
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape, patch_shape2):
    # choose random instances
    #print(dataset.shape[0])
    #print(n_samples)
    ix = randint(0, dataset.shape[0]-1)
    #print(ix)
    # retrieve selected images
    X = dataset[ix:ix+n_samples]
    #print(X.shape)
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape2, 1))
    #print(y.shape)
    return X, y
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape, patch_shape2):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape2, 1))
    #print(y.shape)
	return X, y

# select a batch of real samples
#X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
#X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
# generate a batch of fake samples
#X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
#X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
# update image pool for fake images

In [33]:
import random
from random import random
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			#print(len(pool))
			ix = randint(0, len(pool)-1)
			selected.append(pool[ix])
			pool[ix] = image
	return np.asarray(selected)

In [34]:
def sample_images(trainA,trainB,model_BtoA,model_AtoB,n_patch,n_patch2,epoch):
        c = 3
        #print(trainA.shape)
        imgs_A, _ = generate_real_samples(trainA, 1, n_patch, n_patch2)
        imgs_B, _ = generate_real_samples(trainB, 1, n_patch, n_patch2)
        #print(imgs_A.shape)
        # Translate images to the other domain
        fake_B, _ = generate_fake_samples(model_AtoB, imgs_A, n_patch, n_patch2)
        fake_A, _ = generate_fake_samples(model_BtoA, imgs_B, n_patch, n_patch2)
        # Translate back to original domain
        reconstr_A, _ = generate_fake_samples(model_BtoA, fake_B, n_patch, n_patch2)
        reconstr_B, _ = generate_fake_samples(model_AtoB, fake_A, n_patch, n_patch2)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig1, axs1 = plt.subplots(c)

        cnt = 0
        for j in range(c):
            axs1[j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs1[j].set_title(titles[j])
            axs1[j].axis('off')
            cnt += 1
        fig1.savefig("images_cycle/AtoB/%d.png" % epoch)
        plt.close()

        fig2, axs2 = plt.subplots(c)
        cnt = 3
        for i in range(c):
            axs2[i].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs2[i].set_title(titles[i])
            axs2[i].axis('off')
            cnt += 1
        fig2.savefig("images_cycle/BtoA/%d.png" % epoch)
        plt.close()


In [35]:
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
    # define properties of the training run
    n_epochs, n_batch, = 100, 1
    # determine the output square shape of the discriminator
    #print(d_model_A.output_shape)
    n_patch = d_model_A.output_shape[1]
    n_patch2 = d_model_A.output_shape[2]
    # unpack dataset
    trainA, trainB = dataset
    #print(trainA.shape)
    #print(trainB.shape)
    # prepare image pool for fakes
    poolA, poolB = list(), list()
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch, n_patch2)
        X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch, n_patch2)
        # generate a batch of fake samples
        X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch, n_patch2)
        X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch, n_patch2)
        # update fakes from pool
        X_fakeA = update_image_pool(poolA, X_fakeA)
        X_fakeB = update_image_pool(poolB, X_fakeB)
        # update generator B->A via adversarial and cycle loss
        g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
        # update discriminator for A -> [real/fake]
        dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
        dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
        # update generator A->B via adversarial and cycle loss
        g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
        # update discriminator for B -> [real/fake]
        dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
        dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
        # summarize performance
        print('>%d, dA[loss1:%.3f,loss2:%.3f] dB[loss1:%.3f,loss2:%.3f] g[loss1:%.3f,loss2:%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
        # If at save interval => save generated image samples
        if i % 100 == 0:
          sample_images(trainA,trainB,g_model_BtoA,g_model_AtoB,n_patch,n_patch2,(i/100))
          d_model_A.save('saved_model/p2hDiscA_2_%d.h5' % (i/100))
          d_model_B.save('saved_model/p2hDiscB_2_%d.h5' % (i/100))
          g_model_AtoB.save('saved_model/p2hGenAB_2_%d.h5' % (i/100))
          g_model_BtoA.save('saved_model/p2hGenBA_2_%d.h5' % (i/100))
          print('model saved')

In [116]:
import cv2
def load_imgs():
    # load Printed dataset
    printed_dir = 'datasets2/words/trainA/'
    printed_imgs = list()
    for f in os.listdir(printed_dir):
      #load the image
      #img = image.imread(printed_dir + '/' + f,'r').convert('L')
      img = cv2.imread(printed_dir + '/' + f, cv2.IMREAD_GRAYSCALE)
      #Resize to same as hw_imgs
      dim = (256,64)
      img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
      printed_imgs.append(img)
    # load HW dataset
    hw_dir = 'datasets2/words/trainB/'
    hw_imgs = list()
    for f in os.listdir(hw_dir):
      #load the image
      img = cv2.imread(hw_dir + '/' + f, cv2.IMREAD_GRAYSCALE)
      #Resize to same as hw_imgs
      dim = (256,64)
      img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
      hw_imgs.append(img)
    # load printed test dataset
    printed2_dir = 'datasets/sentences/testA/'
    printed2_imgs = list()
    for f in os.listdir(printed2_dir):
      #load the image
      img = image.imread(printed2_dir + '/' + f,'r')
      printed2_imgs.append(img)
    # load HW test dataset
    hw_dir2 = 'datasets/sentences/testB/'
    hw_imgs2 = list()
    for f in os.listdir(hw_dir2):
      #load the image
      img2 = image.imread(hw_dir2 + '/' + f,'r')
      hw_imgs2.append(img2)
    printed_imgs = np.asarray(printed_imgs).astype('float32') / 255
    hw_imgs = np.asarray(hw_imgs).astype('float32') / 255
    hw_imgs2 = np.asarray(hw_imgs2).astype('float32') / 255
    printed2_imgs = np.asarray(printed2_imgs).astype('float32') / 255
    printed_imgs = printed_imgs.reshape(printed_imgs.shape[0],
                                      printed_imgs.shape[1],
                                      printed_imgs.shape[2],
                                      1)
    hw_imgs = hw_imgs.reshape(hw_imgs.shape[0],
                                      hw_imgs.shape[1],
                                      hw_imgs.shape[2],
                                      1)
    printed2_imgs = printed2_imgs.reshape(printed2_imgs.shape[0],
                                      printed2_imgs.shape[1],
                                      printed2_imgs.shape[2],
                                      1)
    hw_imgs2 = hw_imgs2.reshape(hw_imgs2.shape[0],
                                      hw_imgs2.shape[1],
                                      hw_imgs2.shape[2],
                                      1)

    return printed_imgs, hw_imgs, printed2_imgs, hw_imgs2

In [117]:
# load a dataset as a list of two numpy arrays
trainA, trainB,testA, testB = load_imgs()
print(trainA.shape)
print(trainB.shape)
print(testB.shape)
dataset = (trainA, trainB)

(186, 64, 256, 1)
(500, 64, 256, 1)
(60, 48, 512, 1)


In [ ]:
%%time
# load a dataset as a list of two numpy arrays
trainA, trainB,testA, testB = load_imgs()
print(trainA.shape)
print(trainB.shape)
print(testB.shape)
dataset = (trainA, trainB)

# input shape
image_shape = (trainA.shape[1],trainA.shape[2],1)
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
g_model_AtoB.summary()
g_model_BtoA.summary()
d_model_A.summary()
d_model_B.summary()
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

(186, 64, 256, 1)
(500, 64, 256, 1)
(60, 48, 512, 1)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           (None, 64, 256, 1)   0                                            
__________________________________________________________________________________________________
conv2d_281 (Conv2D)             (None, 64, 256, 32)  1600        input_41[0][0]                   
__________________________________________________________________________________________________
instance_normalization_281 (Ins (None, 64, 256, 32)  64          conv2d_281[0][0]                 
__________________________________________________________________________________________________
activation_151 (Activation)     (None, 64, 256, 32)  0           instance_normalization_281[0][0] 
________________________________________________________

C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Ellysah\Anaconda3\envs\keras-gpu\lib\site-packages\keras\engi

>1, dA[loss1:0.811,loss2:0.297] dB[loss1:1.912,loss2:0.299] g[loss1:27.521,loss2:26.221]
model saved
>2, dA[loss1:0.995,loss2:0.438] dB[loss1:0.982,loss2:0.289] g[loss1:24.476,loss2:24.784]
>3, dA[loss1:1.253,loss2:0.349] dB[loss1:0.806,loss2:0.558] g[loss1:24.635,loss2:24.235]
>4, dA[loss1:0.200,loss2:0.230] dB[loss1:0.675,loss2:1.085] g[loss1:23.764,loss2:23.650]
>5, dA[loss1:1.110,loss2:0.274] dB[loss1:0.534,loss2:1.422] g[loss1:23.610,loss2:23.319]
>6, dA[loss1:1.240,loss2:0.405] dB[loss1:0.676,loss2:1.748] g[loss1:21.648,loss2:23.348]
>7, dA[loss1:0.446,loss2:0.529] dB[loss1:0.401,loss2:0.870] g[loss1:20.721,loss2:21.584]
>8, dA[loss1:0.206,loss2:0.356] dB[loss1:0.196,loss2:0.432] g[loss1:20.983,loss2:21.182]
>9, dA[loss1:0.125,loss2:0.194] dB[loss1:0.260,loss2:0.211] g[loss1:19.679,loss2:20.541]
>10, dA[loss1:0.103,loss2:0.315] dB[loss1:0.417,loss2:0.152] g[loss1:20.962,loss2:20.596]
>11, dA[loss1:0.157,loss2:0.236] dB[loss1:0.297,loss2:0.216] g[loss1:20.289,loss2:20.495]
>12, dA

>92, dA[loss1:0.025,loss2:0.054] dB[loss1:0.033,loss2:0.029] g[loss1:19.049,loss2:18.810]
>93, dA[loss1:0.031,loss2:0.077] dB[loss1:0.046,loss2:0.013] g[loss1:18.565,loss2:18.824]
>94, dA[loss1:0.020,loss2:0.031] dB[loss1:0.034,loss2:0.044] g[loss1:19.234,loss2:19.566]
>95, dA[loss1:0.011,loss2:0.034] dB[loss1:0.072,loss2:0.055] g[loss1:18.708,loss2:19.103]
>96, dA[loss1:0.021,loss2:0.049] dB[loss1:0.032,loss2:0.050] g[loss1:18.041,loss2:18.334]
>97, dA[loss1:0.011,loss2:0.121] dB[loss1:0.054,loss2:0.035] g[loss1:17.968,loss2:18.488]
>98, dA[loss1:0.010,loss2:0.032] dB[loss1:0.044,loss2:0.060] g[loss1:18.372,loss2:18.776]
>99, dA[loss1:0.016,loss2:0.024] dB[loss1:0.059,loss2:0.094] g[loss1:17.561,loss2:18.709]
>100, dA[loss1:0.025,loss2:0.044] dB[loss1:0.060,loss2:0.153] g[loss1:16.608,loss2:17.510]
>101, dA[loss1:0.014,loss2:0.058] dB[loss1:0.060,loss2:0.104] g[loss1:18.281,loss2:18.665]
model saved
>102, dA[loss1:0.014,loss2:0.032] dB[loss1:0.075,loss2:0.023] g[loss1:16.455,loss2:17.

>182, dA[loss1:0.014,loss2:0.020] dB[loss1:0.055,loss2:0.071] g[loss1:17.337,loss2:17.899]
>183, dA[loss1:0.005,loss2:0.011] dB[loss1:0.025,loss2:0.039] g[loss1:17.714,loss2:18.420]
>184, dA[loss1:0.009,loss2:0.025] dB[loss1:0.015,loss2:0.031] g[loss1:16.588,loss2:17.389]
>185, dA[loss1:0.016,loss2:0.032] dB[loss1:0.029,loss2:0.103] g[loss1:18.176,loss2:17.923]
>186, dA[loss1:0.019,loss2:0.012] dB[loss1:0.022,loss2:0.375] g[loss1:18.488,loss2:17.989]
>187, dA[loss1:0.010,loss2:0.021] dB[loss1:0.033,loss2:0.141] g[loss1:16.868,loss2:17.240]
>188, dA[loss1:0.020,loss2:0.057] dB[loss1:0.019,loss2:0.089] g[loss1:18.711,loss2:18.490]
>189, dA[loss1:0.020,loss2:0.021] dB[loss1:0.033,loss2:0.066] g[loss1:17.535,loss2:17.796]
>190, dA[loss1:0.012,loss2:0.024] dB[loss1:0.025,loss2:0.025] g[loss1:17.900,loss2:18.190]
>191, dA[loss1:0.009,loss2:0.063] dB[loss1:0.071,loss2:0.021] g[loss1:18.639,loss2:18.963]
>192, dA[loss1:0.011,loss2:0.041] dB[loss1:0.037,loss2:0.021] g[loss1:17.993,loss2:18.985]

>272, dA[loss1:0.020,loss2:0.018] dB[loss1:0.012,loss2:0.030] g[loss1:17.681,loss2:17.959]
>273, dA[loss1:0.007,loss2:0.007] dB[loss1:0.015,loss2:0.037] g[loss1:17.780,loss2:17.840]
>274, dA[loss1:0.005,loss2:0.027] dB[loss1:0.021,loss2:0.061] g[loss1:17.170,loss2:17.555]
>275, dA[loss1:0.012,loss2:0.121] dB[loss1:0.018,loss2:0.018] g[loss1:16.561,loss2:17.127]
>276, dA[loss1:0.030,loss2:0.792] dB[loss1:0.019,loss2:0.079] g[loss1:16.654,loss2:18.663]
>277, dA[loss1:0.179,loss2:2.089] dB[loss1:0.031,loss2:0.044] g[loss1:17.927,loss2:20.292]
>278, dA[loss1:0.352,loss2:0.239] dB[loss1:0.032,loss2:0.017] g[loss1:17.943,loss2:18.951]
>279, dA[loss1:0.076,loss2:0.055] dB[loss1:0.021,loss2:0.016] g[loss1:18.014,loss2:18.149]
>280, dA[loss1:0.062,loss2:0.049] dB[loss1:0.023,loss2:0.014] g[loss1:16.522,loss2:17.092]
>281, dA[loss1:0.050,loss2:0.030] dB[loss1:0.022,loss2:0.048] g[loss1:18.950,loss2:19.320]
>282, dA[loss1:0.016,loss2:0.045] dB[loss1:0.013,loss2:0.052] g[loss1:16.425,loss2:17.190]

>362, dA[loss1:0.007,loss2:0.006] dB[loss1:0.013,loss2:0.028] g[loss1:17.085,loss2:17.318]
>363, dA[loss1:0.005,loss2:0.003] dB[loss1:0.016,loss2:0.013] g[loss1:17.089,loss2:17.285]
>364, dA[loss1:0.002,loss2:0.008] dB[loss1:0.011,loss2:0.018] g[loss1:16.716,loss2:17.211]
>365, dA[loss1:0.006,loss2:0.021] dB[loss1:0.011,loss2:0.027] g[loss1:16.287,loss2:16.912]
>366, dA[loss1:0.004,loss2:0.026] dB[loss1:0.011,loss2:0.018] g[loss1:17.517,loss2:17.414]
>367, dA[loss1:0.007,loss2:0.012] dB[loss1:0.013,loss2:0.009] g[loss1:16.701,loss2:17.442]
>368, dA[loss1:0.004,loss2:0.010] dB[loss1:0.005,loss2:0.023] g[loss1:15.255,loss2:16.072]
>369, dA[loss1:0.011,loss2:0.003] dB[loss1:0.017,loss2:0.043] g[loss1:17.179,loss2:17.460]
>370, dA[loss1:0.002,loss2:0.008] dB[loss1:0.010,loss2:0.009] g[loss1:16.855,loss2:17.187]
>371, dA[loss1:0.003,loss2:0.002] dB[loss1:0.019,loss2:0.041] g[loss1:16.620,loss2:16.783]
>372, dA[loss1:0.003,loss2:0.002] dB[loss1:0.015,loss2:0.014] g[loss1:16.755,loss2:17.032]

>452, dA[loss1:0.004,loss2:0.003] dB[loss1:0.004,loss2:0.004] g[loss1:14.986,loss2:15.811]
>453, dA[loss1:0.002,loss2:0.003] dB[loss1:0.008,loss2:0.005] g[loss1:16.380,loss2:16.535]
>454, dA[loss1:0.001,loss2:0.005] dB[loss1:0.005,loss2:0.005] g[loss1:17.050,loss2:17.016]
>455, dA[loss1:0.003,loss2:0.002] dB[loss1:0.004,loss2:0.003] g[loss1:15.798,loss2:17.064]
>456, dA[loss1:0.003,loss2:0.004] dB[loss1:0.004,loss2:0.030] g[loss1:14.579,loss2:15.473]
>457, dA[loss1:0.003,loss2:0.008] dB[loss1:0.008,loss2:0.022] g[loss1:16.973,loss2:16.911]
>458, dA[loss1:0.001,loss2:0.015] dB[loss1:0.006,loss2:0.015] g[loss1:16.926,loss2:17.371]
>459, dA[loss1:0.002,loss2:0.018] dB[loss1:0.004,loss2:0.022] g[loss1:14.926,loss2:15.907]
>460, dA[loss1:0.007,loss2:0.010] dB[loss1:0.007,loss2:0.008] g[loss1:16.913,loss2:16.952]
>461, dA[loss1:0.008,loss2:0.004] dB[loss1:0.010,loss2:0.013] g[loss1:15.827,loss2:16.209]
>462, dA[loss1:0.001,loss2:0.016] dB[loss1:0.013,loss2:0.005] g[loss1:17.158,loss2:17.378]

In [26]:
import tensorflow as tf
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.models import load_model
model = load_model('saved_model/p2hGenAB_2_150.h5',custom_objects={'InstanceNormalization':InstanceNormalization})

In [67]:
from text import str_to_image
response = input("Please enter a sentence: ")
im = str_to_image(response)

Please enter a sentence: good morning


In [68]:
import cv2
import tensorflow as tf
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.models import load_model
model = load_model('saved_model/p2hGenAB_2_357.h5',custom_objects={'InstanceNormalization':InstanceNormalization})
#model.saveAtoB(im, 'test_arch1.png')
X = model.predict(im)
#X = np.asarray(X).astype('float32') / 255
print(X.shape)
# This part only for show the image on the screen, you can delete it if you want
#pathA = 'test_arch1.png'
#img = cv2.imread(pathA)
#cv2.imshow('image', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

(1, 48, 512, 1)


In [69]:
#X = np.asarray(X).astype('float32') / 255
cv2.imshow('image',X[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'datasets2/words/trainA/'